In [1]:
# libraries 

from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
import time
from selenium.webdriver.chrome.service import Service

In [2]:
s = Service(r"C:\Users\Hebert Palmera\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")

driver = webdriver.Chrome(service=s)

driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3835760748&distance=25&f_AL=true&f_JT=F&f_WT=2%2C1%2C3&geoId=101318387&keywords=operations%20analyst&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&sortBy=R")

time.sleep(200)


with open('job_listing.csv','w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Job Title', 'Company Name', 'Address', 'Job Link', 'Salary']) #header row
    for page in range(2,4):
        jobs_listing = driver.find_elements(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/ul/li')
        for listing in jobs_listing:
            job_title_element = listing.find_element(By.TAG_NAME, 'strong')
            job_title = job_title_element.text

            company_element = listing.find_element(By.CLASS_NAME, 'job-card-container__primary-description ')
            company = company_element.text

            address_element = listing.find_element(By.CLASS_NAME, 'job-card-container__metadata-item ')
            address = address_element.text

                # Check if job link is available
            try:
                link_element = listing.find_element(By.CLASS_NAME, 'job-card-container__link')
                job_link = link_element.get_attribute('href')
            except:
                job_link = "N/A"
            
            try:
                salary_element = listing.find_element(By.CSS_SELECTOR, 'li.job-card-container__metadata-item:nth-child(2)')
                salary = address_element.text
            except:
                salary = "N/A"
            
            # scroll down for each job element
            driver.execute_script("arguments[0].scrollIntoView();", listing)



            print("Job Title:", job_title)
            print("Company:", company)
            print("Address:", address)
            print("Salary:", salary)
            print("Link:", job_link)
            print("-" * 50)

            csv_writer.writerow([job_title, company, address,job_link,salary])
        driver.find_element(By.XPATH, f"//button[@aria-label='Página {page}']").click()
        time.sleep(3)

driver.quit()

Job Title: DSD Operations Specialist
Company: PRIME
Address: Estados Unidos (En remoto)
Salary: N/A
Link: https://www.linkedin.com/jobs/view/3835760748/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=iZ9tCN7KmK457ITWo7sX9w%3D%3D&trackingId=UmHI42gKxcW%2F%2F3TUg74sJw%3D%3D&trk=flagship3_search_srp_jobs
--------------------------------------------------
Job Title: Lead Operations Analyst
Company: Second Nature
Address: Estados Unidos (En remoto)
Salary: N/A
Link: https://www.linkedin.com/jobs/view/3806068386/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=iZ9tCN7KmK457ITWo7sX9w%3D%3D&trackingId=UiaQf0YbezqWbbJcPI7QLg%3D%3D&trk=flagship3_search_srp_jobs
--------------------------------------------------
Job Title: Strategic Operations Analyst
Company: Cohere Health
Address: Estados Unidos (En remoto)
Salary: N/A
Link: https://www.linkedin.com/jobs/view/3834742385/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=iZ9tCN7KmK457ITWo7sX9w%3D%3D&trackingId=HYtoy78jmIaLJbh%2BiNDQuA%3D%3D&trk=flagship3_search_srp_jobs
-----------

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@aria-label='Página 3']"}
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF72BAB7012+3522402]
	(No symbol) [0x00007FF72B6D8352]
	(No symbol) [0x00007FF72B585ABB]
	(No symbol) [0x00007FF72B5CBF0E]
	(No symbol) [0x00007FF72B5CC08C]
	(No symbol) [0x00007FF72B60E437]
	(No symbol) [0x00007FF72B5EF09F]
	(No symbol) [0x00007FF72B60BDA3]
	(No symbol) [0x00007FF72B5EEE03]
	(No symbol) [0x00007FF72B5BF4D4]
	(No symbol) [0x00007FF72B5C05F1]
	GetHandleVerifier [0x00007FF72BAE9B9D+3730157]
	GetHandleVerifier [0x00007FF72BB3F02D+4079485]
	GetHandleVerifier [0x00007FF72BB375D3+4048163]
	GetHandleVerifier [0x00007FF72B80A649+718233]
	(No symbol) [0x00007FF72B6E4A3F]
	(No symbol) [0x00007FF72B6DFA94]
	(No symbol) [

In [6]:
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

# Path to ChromeDriver executable
chromedriver_path = r"C:\Users\Hebert Palmera\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# Initialize ChromeDriver
s = Service(chromedriver_path)
driver = webdriver.Chrome(service=s)

# Read CSV file containing job details and URLs
df = pd.read_csv('job_listing.csv')

# Function to extract expected salary from URL
def extract_expected_salary(url):
    max_attempts = 3
    for attempt in range(max_attempts):
        time.sleep(3)
        try:
            driver.get(url)
            expected_salary_element = driver.find_element(By.CLASS_NAME, 'salary.compensation__salary')
            expected_salary = expected_salary_element.text
            return expected_salary
        except NoSuchElementException:
            print(f"Attempt {attempt + 1}: Expected salary not found. Retrying...")
            time.sleep(2)  # Pause for a moment before retrying

    return "N/A"

# Extract expected salary for each job listing
df['Expected Salary'] = df['Job Link'].apply(extract_expected_salary)

# Save updated job details to CSV
df.to_csv('job_details_with_salary.csv', index=False)

# Close the ChromeDriver session
driver.quit()

C:\Users\Hebert Palmera\AppData\Local\Temp\ipykernel_26932\2724599727.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Attempt 1: Expected salary not found. Retrying...
Attempt 2: Expected salary not found. Retrying...
Attempt 3: Expected salary not found. Retrying...
Attempt 1: Expected salary not found. Retrying...
Attempt 2: Expected salary not found. Retrying...
Attempt 3: Expected salary not found. Retrying...
Attempt 1: Expected salary not found. Retrying...
Attempt 2: Expected salary not found. Retrying...
Attempt 3: Expected salary not found. Retrying...
Attempt 1: Expected salary not found. Retrying...
Attempt 1: Expected salary not found. Retrying...
Attempt 2: Expected salary not found. Retrying...
Attempt 3: Expected salary not found. Retrying...
Attempt 1: Expected salary not found. Retrying...
Attempt 2: Expected salary not found. Retrying...
Attempt 3: Expected salary not found. Retrying...
Attempt 1: Expected salary not found. Retrying...
Attempt 2: Expected salary not found. Retrying...
Attempt 3: Expected salary not found. Retrying...
Attempt 1: Expected salary not found. Retrying...
